In [ ]:
import time

In [1]:
MODEL_NAME = "premier_league"
MODEL_VERSION = "v1.0"
MODEL_TYPE = "win_outcome"

In [3]:
# UPDATING THE GITHUB OF THE DATASETS
!git add -A
!git commit -m "updates: {MODEL_NAME}-{MODEL_VERSION}-{MODEL_TYPE}: bot commit - {time.time()}"
!git push

[main 501bf33] updates: premier_league-v1.0-win_outcome: bot commit
 1 file changed, 13 insertions(+), 14 deletions(-)
Enumerating objects: 20, done.
Counting objects: 100% (20/20), done.
Delta compression using up to 2 threads
Compressing objects: 100% (15/15), done.
Writing objects: 100% (15/15), 8.98 KiB | 2.24 MiB/s, done.
Total 15 (delta 5), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (5/5), completed with 2 local objects.
To https://github.com/lebyanelm/neural-trained-models.git
   ad7fa25..501bf33  main -> main
